TODO: 
    
    - separar en carga raw, manipulacion, eda
    - no hacer ese fill na cuando paso bool a int (astype?)
    - sacar las listas temporales de cols a elegir y quedarme con cols
    - checkear el astype int .... tema bools: creo que deberia crear la de oficial store y dejarlas como tipo object

In [1]:
# TODO: checkear y cambiar tipos antes de persistir df final?
# cambiar ahi el boolean a int de las cols que quiero,!!!

In [2]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import joblib

In [3]:
os.chdir("..") 

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None


In [5]:
data_path = "data/"

In [6]:
input_path = data_path + "input/"

## Load "raw" csv

In [7]:
df = pd.read_csv(input_path + 'raw_data.csv', sep='\t', encoding='utf-8', )  
df.head()

c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (23,99,100,101,102,109,110,116,117,118,120,121,122,123,124,125,126,127,130,131) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,stop_time,condition,permalink,thumbnail,thumbnail_id,accepts_mercadopago,attributes,original_price,category_id,official_store_id,domain_id,catalog_product_id,tags,catalog_listing,use_thumbnail_id,order_backend,seller.id,seller.permalink,seller.registration_date,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.eshop.nick_name,seller.eshop.eshop_rubro,seller.eshop.eshop_id,seller.eshop.eshop_locations,seller.eshop.site_id,seller.eshop.eshop_logo_url,seller.eshop.eshop_status_id,seller.eshop.seller,seller.eshop.eshop_experience,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.claims.value,seller.seller_reputation.metrics.claims.period,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.delayed_handling_time.value,seller.seller_reputation.metrics.delayed_handling_time.period,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.cancellations.value,seller.seller_reputation.metrics.cancellations.period,seller.seller_reputation.level_id,prices.id,prices.prices,prices.presentation.display_currency,prices.payment_method_prices,installments.quantity,installments.amount,installments.rate,installments.currency_id,address.state_id,address.state_name,address.city_id,address.city_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller_address.id,seller_address.comment,seller_address.address_line,seller_address.zip_code,seller_address.country.id,seller_address.country.name,seller_address.state.id,seller_address.state.name,seller_address.city.id,seller_address.city.name,seller_address.latitude,seller_address.longitude,differential_pricing.id,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_rate,seller.seller_reputation.metrics.delayed_handling_time.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.car_dealer_logo,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,root_category_id,root_category_name,prices,installments,address.area_code,address.phone1,seller_contact.contact,seller_contact.other_info,seller_contact.area_code,seller_contact.phone,seller_contact.area_code2,seller_contact.phone2,seller_contact.email,seller_contact.webpage,location.address_line,location.zip_code,location.subneighborhood,location.neighborhood.id,location.neighborhood.name,location.city.id,location.city.name,location.state.id,location.state.name,location.country.id,location.country.name,location.latitude,location.longitude,seller.home_image_url,sale_terms,discount,category_name,categories_from_root
0,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,NaN,ARS,1,22,buy_it_now,gold_special,2040-12-17T04:00:00.000Z,new,https://www.mercadolibre.com.ar/estereo-para-a...,http://http2.mlstatic.com/D_960260-MLA33082733...,960260-MLA33082733743_122019,True,"[{'source': 1, 'name': 'Marca del stereo', 'va...",NaN,MLA6867,NaN,MLA-VEHICLE_STEREOS,MLA14983192,"['brand_verified', 'extended_warranty_eligible...",True,True,1,8911722,http://perfil

## Choose a subset of columns and persist final df

In [8]:
cols = ['installments.quantity',
 'installments.rate',
 'id',
 'site_id',
 'title',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'condition',
 'accepts_mercadopago',
 'category_id',
 'official_store_id',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'seller.id',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.sales.period',
 'seller.seller_reputation.metrics.sales.completed',
 'seller.seller_reputation.metrics.cancellations.rate',
 'address.state_name',
 'shipping.free_shipping',
 'shipping.mode',
 'shipping.tags',
 'shipping.logistic_type',
 'shipping.store_pick_up',
 'seller.seller_reputation.real_level',
 'root_category_id',
 'root_category_name',
 'discount',
 'category_name',
 'categories_from_root']

In [9]:
ids = [
'id',
'site_id',
'title',
'category_id',
'seller.id',
'catalog_product_id',
'root_category_id',
'root_category_name',
'category_name',
]
joblib.dump(ids, data_path + 'columns/id_cols.joblib')

['data/columns/id_cols.joblib']

In [10]:
len(cols)

47

## df_final

In [11]:
df_final = df.copy()[cols]  
df_final.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,official_store_id,catalog_product_id,tags,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,address.state_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name,categories_from_root
0,12.0,70.17,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,ARS,1,22,buy_it_now,gold_special,new,True,MLA6867,NaN,MLA14983192,"['brand_verified', 'extended_warranty_eligible...",True,8911722,False,False,"['normal', 'user_info_verified', 'eshop', 'cre...",415.0,30.0,historic,0.01,0.99,0.00,385.0,silver,0.0000,0.0000,60 days,68.0,0.0000,Buenos Aires,True,me2,"['self_service_in', 'mandatory_free_shipping']",drop_off,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Estéreos,"['Accesorios para Vehículos', 'Audio para Vehí..."
1,12.0,70.17,MLA869356478,MLA,Neumático Pirelli Formula Energy 185/60 R14 82h,10250.0,ARS,63,1480,buy_it_now,gold_special,new,True,MLA22195,143.0,MLA15000133,"['brand_verified', 'good_quality_picture', 'go...",True,116026826,False,False,"['brand', 'user_info_verified', 'large_seller'...",22539.0,975.0,historic,0.01,0.96,0.03,21564.0,platinum,0.0039,0.0559,60 days,3662.0,0.0046,Buenos Aires,True,me2,['mandatory_free_shipping'],cross_docking,True,NaN,MLA5725,Accesorios para Vehículos,NaN,Neumáticos de Auto y Camioneta,"['Accesorios para Vehículos', 'Neumáticos', 0 ..."
2,12.0,70.17,MLA851109741,MLA,Carcasa Llave Navaja Peugeot 2 Bot 307 308 C/...,799.0,ARS,50,500,buy_it_now,gold_special,new,True,MLA373345,NaN,NaN,"['good_quality_picture', 'good_quality_thumbna...",NaN,435859328,False,False,"['normal', 'user_info_verified', 'eshop', 'msh...",7371.0,386.0,historic,0.01,0.98,0.01,6985.0,platinum,0.0058,0.0033,60 days,2313.0,0.0000,Santa Fe,False,me2,['fulfillment'],fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Llaves Telecomando,"['Accesorios para Vehículos', 'Repuestos Autos..."
3,9.0,0.00,MLA836983125,MLA,Funda Cubre Rueda Auxilio Ford Ecosport,891.0,ARS,100,250,buy_it_now,gold_pro,new,True,MLA403345,NaN,NaN,"['brand_verified', 'good_quality_picture', 'go...",NaN,20861971,False,False,"['normal', 'user_info_verified', 'credits_prof...",1036.0,31.0,historic,0.01,0.99,0.00,1005.0,gold,0.0000,0.0000,60 days,455.0,0.0000,Capital Federal,False,me2,"['fulfillment', 'self_service_in']",fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,0.1,Fundas Cubre Ruedas,"['Accesorios para Vehículos', 'Accesorios de A..."
4,12.0,70.17,MLA899116906,MLA,Servicio De Envío Express De Telepase Mercado ...,655.0,ARS,500,5000,buy_it_now,gold_special,new,True,MLA455216,2422.0,NaN,"['brand_verified', 'good_quality_picture', 'go...",NaN,679055037,False,False,"['brand', 'user_info_verified', 'large_seller'...",25361.0,1525.0,historic,0.03,0.96,0.01,23836.0,NaN,0.0017,0.0000,60 days,12476.0,0.0000,Capital Federal,True,me2,"['fulfillment', 'self_service_in']",fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Dispositivos para TelePASE,"['Accesorios para Vehículos', 0 ..."


In [12]:
df_final.dtypes

installments.quantity                                          float64
installments.rate                                              float64
id                                                              object
site_id                                                         object
title                                                           object
price                                                          float64
currency_id                                                     object
available_quantity                                               int64
sold_quantity                                                    int64
buying_mode                                                     object
listing_type_id                                                 object
condition                                                       object
accepts_mercadopago                                               bool
category_id                                                     object
offici

## Add column

official_store_id boolean

In [13]:
df_final['official_store_id'].isna().any()

True

In [14]:
df_final['official_store_id']

0           NaN
1         143.0
2           NaN
3           NaN
4        2422.0
          ...  
30902       NaN
30903       NaN
30904       NaN
30905       NaN
30906       NaN
Name: official_store_id, Length: 30907, dtype: float64

In [15]:
df_final['official_store_id'] = ~df_final['official_store_id'].isna() 


In [16]:
df_final['official_store_id'].head()

0    False
1     True
2    False
3    False
4     True
Name: official_store_id, dtype: bool

In [17]:
df_final.rename(columns={'official_store_id': 'is_official_store'}, inplace=True)

In [18]:
df_final['is_official_store'].head()

0    False
1     True
2    False
3    False
4     True
Name: is_official_store, dtype: bool

In [19]:
df_final.columns

Index(['installments.quantity', 'installments.rate', 'id', 'site_id', 'title',
       'price', 'currency_id', 'available_quantity', 'sold_quantity',
       'buying_mode', 'listing_type_id', 'condition', 'accepts_mercadopago',
       'category_id', 'is_official_store', 'catalog_product_id', 'tags',
       'catalog_listing', 'seller.id', 'seller.car_dealer',
       'seller.real_estate_agency', 'seller.tags',
       'seller.seller_reputation.transactions.total',
       'seller.seller_reputation.transactions.canceled',
       'seller.seller_reputation.transactions.period',
       'seller.seller_reputation.transactions.ratings.negative',
       'seller.seller_reputation.transactions.ratings.positive',
       'seller.seller_reputation.transactions.ratings.neutral',
       'seller.seller_reputation.transactions.completed',
       'seller.seller_reputation.power_seller_status',
       'seller.seller_reputation.metrics.claims.rate',
       'seller.seller_reputation.metrics.delayed_handling_time

In [20]:
bool_cols = [
'accepts_mercadopago',
'is_official_store',
    # puedo hacerlo como un boolean de si es oficial o no
'catalog_listing',
'seller.car_dealer',
'seller.real_estate_agency',
'shipping.free_shipping',
'shipping.store_pick_up',
]
joblib.dump(bool_cols, data_path + "columns/bool_cols.joblib")

['data/columns/bool_cols.joblib']

In [21]:
df_final = df_final.infer_objects() 

In [23]:
df_final['is_official_store']

0        False
1         True
2        False
3        False
4         True
         ...  
30902    False
30903    False
30904    False
30905    False
30906    False
Name: is_official_store, Length: 30907, dtype: object

In [24]:
df_final['accepts_mercadopago'].isna().any()

False

In [25]:
df_final['catalog_listing'].isna().any()

True

In [26]:
df_final.dtypes 

installments.quantity                                          float64
installments.rate                                              float64
id                                                              object
site_id                                                         object
title                                                           object
price                                                          float64
currency_id                                                     object
available_quantity                                               int64
sold_quantity                                                    int64
buying_mode                                                     object
listing_type_id                                                 object
condition                                                       object
accepts_mercadopago                                             object
category_id                                                     object
is_off

## Manipulate columns

### available quantity

In [27]:
df_final.available_quantity.isna().any()

False

In [28]:
df_final.available_quantity.value_counts().sort_index()

1        17835
2            9
3            6
4            2
5            3
         ...  
3647         1
5000       905
7107         1
7474         1
50000      329
Name: available_quantity, Length: 86, dtype: int64

In [29]:
bins = [0, 50, 100, 150, 200, 250, 500, 5000, 50000, 99999]

In [30]:
labels = [
    1, 50, 100, 150, 200, 250, 500, 5000, 50000,
]

In [31]:
df_final["available_quantity_q"] = pd.cut(df_final['available_quantity'], bins=bins, right=False, labels=labels)  

In [32]:
df_final[[
    "available_quantity",
    "available_quantity_q",
]].head(20)

,available_quantity,available_quantity_q
0,1,1
1,63,50
2,50,50
3,100,100
4,500,500
5,1,1
6,50,50
7,100,100
8,50,50
9,200,200


In [33]:
df_final.available_quantity_q.value_counts().sort_index()

1        17879
50        3528
100       1405
150        987
200        618
250       2003
500       3251
5000       907
50000      329
Name: available_quantity_q, dtype: int64

In [34]:
df_final.available_quantity = df_final.available_quantity_q

In [35]:
df_final.drop('available_quantity_q', axis = 1, inplace=True)

### Target

In [36]:
df_final.sold_quantity.value_counts().sort_index()

0        3153
1         290
2         249
3         230
4         205
         ... 
7137        1
7179        1
8477        1
8568        1
50000      10
Name: sold_quantity, Length: 93, dtype: int64

<p style="color:red; font-weight: bold">Hay valores que no tienen el value esperado (que representa cada categoria) asi que trabajamos eso. </p>

In [37]:
bins = [0, 1, 2, 3, 4, 5, 25, 50, 100, 150, 200, 250, 500, 5000, 50000, 500000]

In [38]:
labels = [
    0, 1, 2, 3, 4, 5, 25, 50, 100, 150, 200, 250, 500, 5000, 50000,
]

In [39]:
df_final["sold_quantity_q"] = pd.cut(df_final['sold_quantity'], bins=bins, right=False, labels=labels)  

In [40]:
df_final[[
    "sold_quantity",
    "sold_quantity_q",
]].head(20)

,sold_quantity,sold_quantity_q
0,22,5
1,1480,500
2,500,500
3,250,250
4,5000,5000
5,500,500
6,500,500
7,500,500
8,500,500
9,500,500


In [41]:
df_final.sold_quantity_q.value_counts().sort_index()

0        3153
1         290
2         249
3         230
4         205
5        2329
25       1835
50       2799
100      2123
150      1722
200      1443
250      4684
500      9128
5000      707
50000      10
Name: sold_quantity_q, dtype: int64

In [42]:
df_final.sold_quantity = df_final.sold_quantity_q

In [43]:
df_final.drop('sold_quantity_q', axis = 1, inplace=True)

### Export final df

In [44]:
file_name = input_path + "data.csv"
df_final.to_csv(file_name, sep='\t', encoding='utf-8', index=False)